<a href="https://colab.research.google.com/github/brendanpshea/database_sql/blob/main/SQL_Select_Quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import sqlite3
import random
from datetime import datetime, timedelta

class SQLQuiz:
    def __init__(self, quiz_db_path='quiz_questions.db', source_db_path='space_academy.db'):
        self.quiz_db_path = quiz_db_path
        self.source_db_path = source_db_path
        self.current_question = 0
        self.questions = []
        self.answers = []

    def create_default_database(self):
        with sqlite3.connect(self.source_db_path) as conn:
            cursor = conn.cursor()

            # Create Students table
            cursor.execute('''
            CREATE TABLE IF NOT EXISTS students (
                student_id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                species TEXT NOT NULL,
                home_planet TEXT NOT NULL,
                admission_date DATE NOT NULL,
                age INTEGER NOT NULL,
                gpa FLOAT NOT NULL
            )
            ''')

            # Create Classes table
            cursor.execute('''
            CREATE TABLE IF NOT EXISTS classes (
                class_id INTEGER PRIMARY KEY,
                class_name TEXT NOT NULL,
                instructor TEXT NOT NULL,
                max_capacity INTEGER NOT NULL,
                credits INTEGER NOT NULL,
                department TEXT NOT NULL
            )
            ''')

            # Create Enrollments table
            cursor.execute('''
            CREATE TABLE IF NOT EXISTS enrollments (
                enrollment_id INTEGER PRIMARY KEY,
                student_id INTEGER,
                class_id INTEGER,
                enrollment_date DATE NOT NULL,
                grade FLOAT,
                FOREIGN KEY (student_id) REFERENCES students (student_id),
                FOREIGN KEY (class_id) REFERENCES classes (class_id)
            )
            ''')

            num_students = random.randint(50, 150)
            num_classes = random.randint(5, 20)

            # Generate and insert random data
            students_data = self.generate_students(num_students)
            cursor.executemany('INSERT INTO students (name, species, home_planet, admission_date, age, gpa) VALUES (?, ?, ?, ?, ?, ?)', students_data)

            classes_data = self.generate_classes(num_classes)
            cursor.executemany('INSERT INTO classes (class_name, instructor, max_capacity, credits, department) VALUES (?, ?, ?, ?, ?)', classes_data)

            enrollments_data = self.generate_enrollments(num_students, num_classes)
            cursor.executemany('INSERT INTO enrollments (student_id, class_id, enrollment_date, grade) VALUES (?, ?, ?, ?)', enrollments_data)

            conn.commit()

    def generate_students(self, num_students):
        first_names = ["Zorp", "Lira", "Blip", "Galax", "Nebula", "Quasar", "Zenith", "Vortex", "Aurora", "Neutron", "Cosmic", "Pulsar", "Gravity", "Quantum", "Stellar"]
        last_names = ["Xylax", "Starwhisper", "Neutron", "Stormrider", "Moonshadow", "Flux", "Stardust", "Cosmic", "Lightweave", "Starburst"]
        species_list = ["Zorlack", "Elf", "Robot", "Human", "Lunarian", "Energy Being", "Celestial", "Vortexian", "Photonic", "Chronovore"]
        planets = ["Xenon-7", "Eldoria", "Mechanica", "Earth", "Luna", "Novaria", "Astralis", "Whirlpool-9", "Lumina", "Temporia"]

        students = []
        for _ in range(num_students):
            name = f"{random.choice(first_names)} {random.choice(last_names)}"
            species = random.choice(species_list)
            planet = random.choice(planets)
            admission_date = (datetime(2340, 1, 1) + timedelta(days=random.randint(0, 365*10))).strftime('%Y-%m-%d')
            age = random.randint(9, 100)
            gpa = round(random.uniform(2.0, 4.0), 2)
            students.append((name, species, planet, admission_date, age, gpa))

        return students

    def generate_classes(self, num_classes):
        class_names = ["Astro-Navigation", "Xenobiology", "Quantum Mechanics", "Telepathy", "Lightsaber Combat",
                       "Wormhole Engineering", "Alien Linguistics", "Cosmic History", "Zero-G Athletics", "Interstellar Diplomacy"]
        instructors = ["Dr.", "Professor", "Master", "Captain", "Archivist", "Ambassador"]
        departments = ["Navigation", "Biology", "Physics", "Psionics", "Combat", "Engineering", "Languages", "History", "Physical Education", "Politics"]

        classes = []
        for i in range(num_classes):
            name = f"{random.choice(class_names)} {random.randint(101, 999)}"
            instructor = f"{random.choice(instructors)} {random.choice(['Alpha', 'Beta', 'Gamma', 'Delta', 'Epsilon'])}"
            max_capacity = random.randint(15, 50)
            credits = random.randint(1, 5)
            department = random.choice(departments)
            classes.append((name, instructor, max_capacity, credits, department))

        return classes

    def generate_enrollments(self, num_students, num_classes):
        enrollments = []
        for student_id in range(1, num_students + 1):
            num_enrollments = random.randint(1, 5)
            for _ in range(num_enrollments):
                class_id = random.randint(1, num_classes)
                enrollment_date = (datetime(2340, 1, 1) + timedelta(days=random.randint(0, 365*10))).strftime('%Y-%m-%d')
                grade = round(random.uniform(2.0, 4.0), 1) if random.random() > 0.1 else None
                enrollments.append((student_id, class_id, enrollment_date, grade))

        return enrollments

    def add_quiz_question(self, question, answer):
        with sqlite3.connect(self.quiz_db_path) as conn:
            cursor = conn.cursor()
            cursor.execute('INSERT INTO quiz_questions (question, answer) VALUES (?, ?)', (question, answer))
            conn.commit()

    def load_questions(self):
        with sqlite3.connect(self.quiz_db_path) as conn:
            cursor = conn.cursor()
            cursor.execute('SELECT question, answer FROM quiz_questions')
            questions_and_answers = cursor.fetchall()
            self.questions, self.answers = zip(*questions_and_answers)

    def get_table_schemas(self):
        with sqlite3.connect(self.source_db_path) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = cursor.fetchall()
            schemas = []
            for table in tables:
                table_name = table[0]
                cursor.execute(f"PRAGMA table_info({table_name})")
                columns = cursor.fetchall()
                schemas.append((table_name, columns))
        return schemas

    def render_table_schemas(self):
        schemas = self.get_table_schemas()
        schema_html = "<h2>Database Schema:</h2>"
        schema_html += "<ol>"
        for table_name, columns in schemas:
            column_info = ", ".join(f"{column[1]} {column[2]}" for column in columns)
            schema_html += f"<li><b>{table_name}</b> ({column_info})</li>"
        schema_html += "</ol>"

        schema_html += "<h3>Sample queries</h3>"
        sample_query = f'SELECT * FROM {schemas[0][0]} returns all rows and columns from {schemas[0][0]}.'
        schema_html += sample_query + "<br>"
        sample_query = f'SELECT {schemas[0][1][1][1]} FROM {schemas[0][0]} selects a specific column.'
        schema_html += sample_query
        return schema_html


    def create_quiz_database(self):
        """
        Creates a new database to store quiz questions and answers.
        """
        with sqlite3.connect(self.quiz_db_path) as conn:
            cursor = conn.cursor()
            cursor.execute('''
            CREATE TABLE IF NOT EXISTS quiz_questions (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                question TEXT NOT NULL,
                answer TEXT NOT NULL,
                problem_set TEXT NOT NULL
            )
            ''')
            conn.commit()

    def add_quiz_question(self, question, answer, problem_set):
        """
        Adds a new quiz question to the database.
        """
        with sqlite3.connect(self.quiz_db_path) as conn:
            cursor = conn.cursor()
            cursor.execute('INSERT INTO quiz_questions (question, answer, problem_set) VALUES (?, ?, ?)',
                           (question, answer, problem_set))
            conn.commit()

    def load_questions(self, problem_set=None):
        """
        Loads questions from the database, optionally filtered by problem set.
        """
        with sqlite3.connect(self.quiz_db_path) as conn:
            cursor = conn.cursor()
            if problem_set:
                cursor.execute('SELECT question, answer FROM quiz_questions WHERE problem_set = ?', (problem_set,))
            else:
                cursor.execute('SELECT question, answer FROM quiz_questions')
            questions_and_answers = cursor.fetchall()
            self.questions, self.answers = zip(*questions_and_answers) if questions_and_answers else ([], [])

    def run_quiz(self, problem_set=None):
        if not os.path.exists(self.source_db_path):
            self.create_default_database()
        if not os.path.exists(self.quiz_db_path):
            self.create_quiz_database()
            self.add_default_questions()
        self.load_questions(problem_set)

        if not self.questions:
            display(HTML(f"<div>No questions found{' for the specified problem set' if problem_set else ''}.</div>"))
            return

        self.text_area = widgets.Textarea(value='', placeholder='Type your SQL query here...', description='Query:', layout=widgets.Layout(width='60%', height='100px'))
        self.submit_button = widgets.Button(description="Submit")
        self.next_button = widgets.Button(description="Next Question", layout=widgets.Layout(visibility='hidden'))
        self.query_widget = widgets.VBox([self.text_area, self.submit_button, self.next_button])

        self.submit_button.on_click(self.submit_query)
        self.next_button.on_click(self.next_question)

        self.display_current_question()

    def add_default_questions(self):
        default_questions = [
            ("List all student names and their species.", "SELECT name, species FROM students", "basic"),
            ("How many students are enrolled in each class?", "SELECT c.class_name, COUNT(e.student_id) as enrolled_students FROM classes c LEFT JOIN enrollments e ON c.class_id = e.class_id GROUP BY c.class_id", "intermediate"),
            ("What is the average class size?", "SELECT AVG(enrolled_students) as average_class_size FROM (SELECT c.class_id, COUNT(e.student_id) as enrolled_students FROM classes c LEFT JOIN enrollments e ON c.class_id = e.class_id GROUP BY c.class_id)", "intermediate"),
            ("List all classes with their instructors, ordered by class name.", "SELECT class_name, instructor FROM classes ORDER BY class_name", "basic"),
            ("Find the student(s) enrolled in the most classes.", "SELECT s.name, COUNT(e.class_id) as num_classes FROM students s JOIN enrollments e ON s.student_id = e.student_id GROUP BY s.student_id ORDER BY num_classes DESC LIMIT 1", "advanced")
        ]
        for question, answer, problem_set in default_questions:
            self.add_quiz_question(question, answer, problem_set)

    def display_current_question(self):
        clear_output(wait=True)
        display(HTML(self.render_table_schemas()))
        question_html = f"<h3>SQL Question {self.current_question + 1}:</h3><p>{self.questions[self.current_question]}</p>"
        display(HTML(question_html))

        self.text_area.value = ''
        self.submit_button.layout.visibility = 'visible'
        self.next_button.layout.visibility = 'hidden'
        display(self.query_widget)

    def submit_query(self, button):
        user_query = self.text_area.value.strip()

        if not user_query.lower().startswith('select'):
            display(HTML("<div style='color: red;'><strong>Error:</strong> Please enter a valid SELECT query.</div>"))
            return

        try:
            with sqlite3.connect(self.source_db_path) as conn:
                user_result = pd.read_sql_query(user_query, conn)
                correct_query = self.answers[self.current_question]
                correct_result = pd.read_sql_query(correct_query, conn)

            if user_result.equals(correct_result):
                display(HTML("<div style='color: green;'><strong>Correct!</strong> Your query produced the expected result.</div>"))
                self.submit_button.layout.visibility = 'hidden'
                self.next_button.layout.visibility = 'visible'
            else:
                display(HTML("<div style='color: red;'><strong>Incorrect.</strong> Your query did not produce the expected result. Please try again.</div>"))

            display(HTML("<h4>Your Results:</h4>"))
            display(user_result)
            display(HTML("<h4>Expected Results:</h4>"))
            display(correct_result)

        except Exception as e:
            display(HTML(f"<div style='color: red;'><strong>Error:</strong> {str(e)}</div>"))

    def next_question(self, button):
        self.current_question += 1
        if self.current_question < len(self.questions):
            self.display_current_question()
        else:
            self.submit_button.layout.visibility = 'hidden'
            self.next_button.layout.visibility = 'hidden'
            display(HTML("<div>All questions completed. Well done!</div>"))

# Usage example
quiz = SQLQuiz()
quiz.run_quiz()

,name,species
0,Aurora Stormrider,Celestial
1,Vortex Stormrider,Zorlack
2,Quasar Flux,Vortexian
3,Gravity Starburst,Zorlack
4,Lira Xylax,Zorlack
...,...,...
128,Neutron Flux,Photonic
129,Galax Stormrider,Human
130,Stellar Neutron,Energy Being
131,Gravity Starburst,Zorlack


,name,species
0,Aurora Stormrider,Celestial
1,Vortex Stormrider,Zorlack
2,Quasar Flux,Vortexian
3,Gravity Starburst,Zorlack
4,Lira Xylax,Zorlack
...,...,...
128,Neutron Flux,Photonic
129,Galax Stormrider,Human
130,Stellar Neutron,Energy Being
131,Gravity Starburst,Zorlack
